In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import datetime
from IPython.display import clear_output
from datetime import timedelta
from scipy.stats import skewnorm
import inspect, re
import h2o
from h2o.automl import H2OAutoML
import xgboost as xgb
               
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: Picked up _JAVA_OPTIONS: -Xverify:none; java version "12.0.1" 2019-04-16; Java(TM) SE Runtime Environment (build 12.0.1+12); Java HotSpot(TM) 64-Bit Server VM (build 12.0.1+12, mixed mode, sharing)
  Starting server from /anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/0y/vcjmh6651517f16c4qtnxyg00000gq/T/tmpazt8qb7h
  JVM stdout: /var/folders/0y/vcjmh6651517f16c4qtnxyg00000gq/T/tmpazt8qb7h/h2o_josephdown_started_from_python.out
  JVM stderr: /var/folders/0y/vcjmh6651517f16c4qtnxyg00000gq/T/tmpazt8qb7h/h2o_josephdown_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,1 month and 3 days
H2O cluster name:,H2O_from_python_josephdown_bis3r7
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
fy_10 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy10-operating-budget.xlsx")
fy_11 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy11-operating-budget.xlsx")
fy_12 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy12-operating-budget.xlsx")
fy_13 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy13-operating-budget.xlsx")
fy_14 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy14-operating-budget.xlsx")
fy_15 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy15-operating-budget.xlsx")
fy_16 = pd.read_excel("fy16-budget-vs-actuals-revenue-and-expenses.xls")

#Clean up the formating of FY16
fy_16.columns = fy_16.loc[0].values
fy_16 = fy_16.loc[1:]

In [4]:
fy_list = [fy_10, fy_11, fy_12, fy_13, fy_14, fy_15, fy_16]

for year in fy_list:
    year['OA_Difference'] = year['Actuals']-year['Original Budget']
    year = year.loc[year['Revenue or Expenditure']=="Expenditures"]
    year.dropna(inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
df = pd.concat(fy_list)
df.dropna(inplace=True)

houston_data = pd.read_csv("Houston Yearly Data - Sheet1 (1).csv")
houston_data_columns = houston_data.iloc[1,1:].values
houston_data_index = houston_data.iloc[2:,0].values
houston_data  = houston_data.iloc[2:,1:]


In [6]:
houston_data.index= houston_data_index
houston_data.columns = houston_data_columns
houston_data.columns = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019']
houston_data.fillna(0, inplace=True)
houston_data['2019'] = houston_data['2019'].astype(int)



# sum_ = 0
# for column in categorical_variables:
#     print(column, len(df[column].value_counts()))
#     sum_ += len(df[column].value_counts())
# print(sum_)

In [7]:
categorical_variables = ['Fund Name', 
                         'Fund Type Descr', 
                         'Business Area Name', 
                        'Commitment Set Description']

dummies = pd.get_dummies(df[categorical_variables]) # One-hot encoding of categorical variables
df = pd.concat([dummies, df.drop(categorical_variables, axis=1)], axis=1)


In [15]:
fy_10.head()

,Fiscal Year,Fund Id,Fund Name,Fund Type Descr,Business Area,Business Area Name,Fund Center Id,Fund Center Name,GL Account,GL Description,GL Category,GL Category Description,Commitment Set Description,Revenue or Expenditure,Original Budget,Current Budget,Actuals,OA_Difference
0,2010,1000,General Fund,General Funds,1000,Houston Police Department,1000009999,HPD - Unmapped Cost,503040,503040 - Health Insurance - Retiree Classified,500,500 - Personnel Services,50 - Total Expenditures,Expenditures,0.0,0.0,0.00,0.00
1,2010,1000,General Fund,General Funds,1000,Houston Police Department,1000010001,HPD-Chief of Police,500010,500010 - Salary Base Pay - Civilian,500,500 - Personnel Services,50 - Total Expenditures,Expenditures,997461.0,1831618.0,960305.36,-37155.64
2,2010,1000,General Fund,General Funds,1000,Houston Police Department,1000010001,HPD-Chief of Police,500020,500020 - Salary Base Pay - Classified,500,500 - Personnel Services,50 - Total Expenditures,Expenditures,1320119.0,1320119.0,1258680.84,-61438.16
3,2010,1000,General Fund,General Funds,1000,Houston Police Department,1000010001,HPD-Chief of Police,500040,500040 - Salary Assignment Pay - Classified,500,500 - Personnel Services,50 - Total Expenditures,Expenditures,0.0,0.0,0.00,0.00
4,2010,1000,General Fund,General Funds,1000,Houston Police Department,1000010001,HPD-Chief of Police,500050,500050 - Salary-Education/Incentive Pay - Classif,500,500 - Personnel Services,50 - Total Expenditures,Expenditures,75000.0,75000.0,70887.50,-4112.50


In [25]:
def add_houston_data(year_df, houston_data, houston_data_index):
    
    for col in houston_data_index:
        year_df[col] = 0
    
    year_df
    for index, row in year_df.iterrows():
        budget_year = row['Fiscal Year']
        previous_year =  row['Fiscal Year'] - 1
        year_df.loc[index, 'Rockets Playoff Rounds'] = houston_data[str(previous_year)]['Rockets Playoff Rounds']
        year_df.loc[index, 'Rockets Total Attendance'] = houston_data[str(previous_year)]['Rockets Total Attendance']
        year_df.loc[index, 'Texans Average Attendance'] = houston_data[str(budget_year)]['Texans Average Attendance']
        year_df.loc[index, 'Texans Playoff Rounds'] = houston_data[str(budget_year)]['Texans Playoff Rounds']
        year_df.loc[index, 'Astros Yearly Attendance'] = houston_data[str(previous_year)]['Astros Yearly Attendance']
        year_df.loc[index, 'Astros Playoff Rounds'] = houston_data[str(previous_year)]['Astros Playoff Rounds']
        
        year_df.loc[index, 'Yearly Crime Index'] = houston_data[str(previous_year)]['Yearly Crime Index']
        year_df.loc[index, 'Yearly Avg High Temp'] = houston_data[str(previous_year)]['Yearly Avg High Temp']
        year_df.loc[index, 'Yearly Avg Low Temp'] = houston_data[str(previous_year)]['Yearly Avg Low Temp']
        year_df.loc[index, 'Yearly Rain Fall'] = houston_data[str(previous_year)]['Yearly Rain Fall']
        
        

In [35]:
for year in fy_list:
    dummies = pd.get_dummies(year[categorical_variables]) # One-hot encoding of categorical variables
    year = pd.concat([dummies, year.drop(categorical_variables, axis=1)], axis=1)
    year.drop('Fund Center Name', axis=1, inplace=True)
    add_houston_data(year, houston_data, houston_data_index)

In [36]:
train_list = [fy_10, fy_11, fy_12, fy_13, fy_14]
test_list = [fy_15, fy_16]

In [37]:
X_train = pd.concat(train_list).drop(['OA_Difference','Actuals','Current Budget'], axis=1)
X_test = pd.concat(test_list).drop(['OA_Difference', 'Actuals', 'Current Budget'], axis = 1)
y_train = pd.concat(train_list)['OA_Difference']
y_test = pd.concat(test_list)['OA_Difference']


In [38]:
X_train.dtypes

Fiscal Year                     int64
Fund Id                         int64
Fund Name                      object
Fund Type Descr                object
Business Area                   int64
Business Area Name             object
Fund Center Id                  int64
Fund Center Name               object
GL Account                      int64
GL Description                 object
GL Category                     int64
GL Category Description        object
Commitment Set Description     object
Revenue or Expenditure         object
Original Budget               float64
dtype: object

In [33]:
model = xgb.XGBRegressor()
model.fit(X_train,y_train)
print (model)

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields Fund Name, Fund Type Descr, Business Area Name, Fund Center Name, GL Description, GL Category Description, Commitment Set Description, Revenue or Expenditure

AttributeError: module 'xgboost' has no attribute 'fit'

In [ ]:
xgb_test_predictions = xgb.predict(X_test)
xgb_train_predictions = xgb.predict(X_train)

In [ ]:
np.sqrt(np.mean((xgb_test_predictions-y_test)**2))


In [ ]:
x = np.linspace(-100000000,100000000,100000000)
plt.figure(figsize=(12,12))
plt.scatter(y_train, xgb_train_predictions, c='red')
plt.scatter(y_test, xgb_test_predictions, c='blue')
#plt.scatter(train_df['voyage_duration'], train_df['eta_diff'], c='green')
plt.plot(x,x,c='black')
plt.xlim([-5,100000000])
plt.ylim([-5,100000000])
plt.xlabel('observations')
plt.ylabel('predictions')

In [ ]:
fy_name_list = ['fy_10', 'fy_11', 'fy_12', 'fy_13', 'fy_14', 'fy_15', 'fy_16']

for index, year in enmerate(fy_list):
    pd.to_csv(year, (fy_name_list[index] + fy_name_list[index]))

In [14]:
# create h2o frame for training data
training_frame = pd.concat([X_train,y_train], axis=1)
training_frame.columns = list(training_frame.columns[:-1]) + ['OA_Difference']
training_frame = h2o.H2OFrame(training_frame)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
# create h2o frame for testing data
testing_frame = pd.concat([X_test,y_test], axis=1)
testing_frame.columns = list(testing_frame.columns[:-1]) + ['OA_Difference']
testing_frame = h2o.H2OFrame(testing_frame)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(None, 'OA_Difference', training_frame)

In [ ]:
predicted_actuals = aml.predict(h2o.H2OFrame(pd.concat([X_test, y_test], axis=1)))
np.sqrt(np.mean((predicted_actuals['predict'].as_data_frame()['predict'].values-y_test)**2))


In [ ]:
x = np.linspace(-100000000,100000000,100000000)
plt.figure(figsize=(12,12))
plt.scatter(y_train, aml.predict(training_frame).as_data_frame()['predict'].values, c='red')
plt.scatter(y_test, aml.predict(testing_frame).as_data_frame()['predict'].values, c='blue')
#plt.scatter(train_df['voyage_duration'], train_df['eta_diff'], c='green')
plt.plot(x,x,c='black')
plt.xlim([-5,100000000])
plt.ylim([-5,100000000])
plt.xlabel('observations')
plt.ylabel('predictions')


In [22]:
aml.leader

In [ ]:
#2:10pm 8/30

In [19]:
h2o.cluster().shutdown()

H2O session _sid_9752 closed.


In [ ]:
a = np.random.random(50)
print(a)
print(a[::3])

In [ ]:
df['OA_Difference'].describe()

In [ ]:
sns.distplot(a)

In [ ]:
df['OA_Difference'].shape

In [ ]:
len(df.loc[(df['OA_Difference']<=1.0) & (df['OA_Difference']>=-1.0)]['OA_Difference'])

In [ ]:
df['OA_Difference'].value_counts()

In [ ]:
OA_Diff = df['OA_Difference']

In [ ]:
#Seaborn distplot of room 2170 Intro to Finance
sns.set_style('darkgrid')
sns.set(font_scale=2.5)

fig, ax = plt.subplots(figsize=(20,15))
sns.distplot(list(OA_Diff),bins=10, ax=ax)

ax.set_xlim(-8e7,8e7)
#ax.set(xticks=[-40000000, -20000000,0, 20000000, 40000000])

# ax.set_title("32 Different Teachers - Intro to Financing \n Grade Distriubtions Room 2170")
# ax.set_ylabel("Frequency")
# ax.set_xlabel("Percent A's Given")

In [ ]:
sns.distplot(list(OA_Diff))

In [ ]:
df['OA_Difference'].max()

In [ ]:
df['OA_Difference'].std()

In [ ]:
df['OA_Difference'].min()

In [ ]:
df['Original Budget'].std()

In [ ]:
df['Original Budget'].min()

In [ ]:
df.loc[df['Original Budget']==df['Original Budget'].min()]

In [ ]:
(df.loc[df['Original Budget']>0]['OA_Difference'].values).std()

In [ ]:
original_df = df.copy(deep=True)

In [ ]:
df = df.loc[df['Original Budget']>0]

In [ ]:
#plt.hist(df['OA_Difference'].values)

In [ ]:
%%capture --no-display
pandas_profiling.ProfileReport(df)

In [ ]:
plt.hist(df['OA_Difference'].values)